In [58]:
####################################################################
# Imports and given constants and classes to be used in your code
# DO NOT CHANGE ANY OF THE BELOW
####################################################################

####################################################################
# Authors:
# Aristidis Karidis - 230507748
# Harris Ilhan Bin Ahmad Affandi - 231101022
####################################################################

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle 
import math

# UNITS USED
# Time: Astronomical Units (AU). 1AU = distance between Sun and earth
# Mass: Solar Mass (M_sun). 1M_sun = mass of the Sun
# Time: Year (yr). 1yr = 1 year (one rotation of the Earth around the Sun)
# Luminosity: Solar Luminosity (L_sun). 1L_sun = luminosity of the Sun
# Velocity: AU/yr

G = 4*(math.pi)**2  # Universal gravitational constant (AU^3/M_sun/yr^2)
scoeff = 22.62      # Scorching coefficient, approx. 0.745 AU from the Sun (L_sun/AU^2)
fcoeff = 3.14       # Freezing coefficient, approx. 2 AU from the Sun (L_sun/AU^2)

class Box:
    def __init__(self,x0,y0,x1,y1):
        self.x0, self.y0, self.x1, self.y1 = x0, y0, x1, y1
        self.maxSide = max(self.x1-self.x0,self.y1-self.y0)
        self.mx = (self.x0+self.x1)/2
        self.my = (self.y0+self.y1)/2
        
    def isIn(self,p):
        return self.x0 <= p.x <= self.x1 and self.y0 <= p.y <= self.y1
    
    def asTuple(self):
        return (self.x0,self.y0,self.x1,self.y1,self.mx,self.my)
    
    def split4(self):
        bNE = Box(self.mx,self.my,self.x1,self.y1)
        bNW = Box(self.x0,self.my,self.mx,self.y1)
        bSW = Box(self.x0,self.y0,self.mx,self.my)
        bSE = Box(self.mx,self.y0,self.x1,self.my)
        return (bNE,bNW,bSW,bSE)

    # Build a box that encloses all Bodies from an array 
    @staticmethod
    def getBox(P):
        if len(P) == 0: return None
        x0 = x1 = P[0].x
        y0 = y1 = P[0].y
        for p in P: 
            if p.x < x0: x0 = p.x
            if p.y < y0: y0 = p.y
            if p.x > x1: x1 = p.x
            if p.y > y1: y1 = p.y
        return Box(x0,y0,x1,y1)    
    
    def __str__(self):
        return f"Box({self.x0},{self.y0},{self.x1},{self.y1})"
        
class GNode:
    def __init__(self,box):
        self.box = box
        self.COM = None           # this node’s COM
        self.nbodies = 0          # number of bodies contained in node
        self.p = None             # if this node is a leaf with a Body
        self.children = None      # children are: [NE, NW, SW, SE]
        self.updateCOM()

    def isLeaf(self):
        return self.nbodies < 2
        
    def updateCOM(self):
        if self.isLeaf(): 
            if self.p == None:
                self.COM = Body(0, self.box.mx, self.box.my)
            else:
                self.COM = Body(self.p.m, self.p.x, self.p.y)
            return
        x = y = m = 0
        for c in self.children:
            x += c.COM.x*c.COM.m
            y += c.COM.y*c.COM.m
            m += c.COM.m
        self.COM = Body(m, x/m, y/m)

    def niceStr(self): 
        S = ("├","─","└","│")
        angle = S[2]+S[1]+" "
        vdash = S[0]+S[1]+" "
        
        def niceRec(ptr,acc,pre,A):
            if ptr == None: raise Exception("A None GNode was found")
            val = f"{len(A)}:{ptr.box},{ptr.nbodies}"
            A.append(f"({ptr.COM.m}, {ptr.COM.x}, {ptr.COM.y})")
            if ptr.children==None: return acc+pre+val
            if pre == vdash: pre2 = S[3]+"  "
            elif pre == angle: pre2 = "   "
            else: pre2 = ""
            T = [vdash,vdash,vdash,angle]
            for i in range(4):
                T[i] = niceRec(ptr.children[i],acc+pre2,T[i],A)
            return acc+pre+val+"\n"+T[0]+"\n"+T[1]+"\n"+T[2]+"\n"+T[3]
            
        A = []
        s = niceRec(self,"","",A)+"\n"
        for i in range(len(A)):
            s += f"\n{i}{' '*(3-len(str(i)))}-> {A[i]}"
        return s
    
class Stack:
    def __init__(self):
        self.inList = []
        
    def push(self,v):
        self.inList.append(v)

    def pop(self):
        if len(self.inList) == 0: raise Exception("Popping from an empty stack")
        return self.inList.pop()
    
    def isEmpty(self):
        return len(self.inList) == 0
    
    def size(self):
        return len(self.inList)

    def toArray(self):
        return self.inList
    
    def __str__(self):
        return str(self.inList)

In [59]:
####################################################################
# Question 1: Intro to the n-body problem
# To solve: closest distance, predict good eras in 3-body problem
####################################################################

class Body:
    def __init__(self,m,x,y,vx=0,vy=0):
        self.m = m     # mass
        self.x = x     # position
        self.y = y     # position
        self.vx = vx   # velocity
        self.vy = vy   # velocity
        
    def squareDist(self,other):
        return (self.x-other.x)**2+(self.y-other.y)**2

    def __str__(self):
        return f"Body({self.m},{self.x},{self.y},{self.vx},{self.vy})"    

    def __repr__(self):
        return str(self)    

    def asTuple(self):
        return (self.m,self.x,self.y,self.vx,self.vy)    
    
    # Gives the next position and velocity of the current Body to its position
    # and velocity after time dt las elapsed, taking into account the pull
    # forces from the bodies in the array Bodies.
    def next(self, Bodies, dt):
        ret = Body(self.m, self.x, self.y)
        ax = ay = 0       
        # for each p in Bodies we compute their force on this Body and add it to its acceleration 
        for p in Bodies:
            if p == self: continue # current Body does not affect itself          
            # euclidian distance between p and this Body
            sq_distance = self.squareDist(p)
            # see e.g. https://en.wikipedia.org/wiki/Newton%27s_law_of_universal_gravitation
            # for the vector form of Newton's law of gravity            
            ax += (p.x - self.x) *  p.m * G / (sq_distance**1.5)
            ay += (p.y - self.y) *  p.m * G / (sq_distance**1.5)
        # we compute displacement due to acceleration that we computed above
        # and due to its current speed because of inertia        
        ret.x += dt*dt*ax + dt*self.vx
        ret.y += dt*dt*ay + dt*self.vy
        # compute the velocity vectors
        ret.vx = (ret.x-self.x)/dt
        ret.vy = (ret.y-self.y)/dt

        return ret

    # Predict years of stability that self planet is going to have in a 3-sun solar system.
    # It suffices to take into account just the three suns (sunA,sunB,sunC) and the planet.
    # Stability is broken if one of the following criteria is violated: 
    # a. No scorching: lA/dA + lB/dB + lC/dC must be less than scoeff
    # b. No freezing: lA/dA + lB/dB + lC/dC must be greater than fcoeff
    # Notes: 
    # - dA, dB, dC are the squared distances between sunA, sunB, sunC and self, and 
    # - lA, lB, lC are the luminosities of sunA, sunB, sunC 
    # respectively. 
    def threeBodyProblem(self,sunA,sunB,sunC,lA,lB,lC): # 15%
        #note, self is the planet
        #if planets is habitable throughout the simulation, return 10.01, if scorched, return time of scorching, if frozen, return time of freezing, if frozen from start return 0.0
        #seld scoeff should be less than 22.62, or it will scorch
        #fcoeff should be greater than 3.14, or it will freeze
        if self.vx == 0 and self.vy == 0:
            return 0.0
        
        planets_array = [sunA, sunB, sunC, self]
        sim = Simulation(planets_array)
        timesteps = sim.timesteps
        dt = sim.dt
        steps = 0
        for t in range(timesteps):
            
            #get current positions of bodies
            current_bodies = sim.run()[t]
            planet = current_bodies[3]
            sunA_current = current_bodies[0]
            sunB_current = current_bodies[1]
            sunC_current = current_bodies[2]

            #calculate illumination
            dA = planet.squareDist(sunA_current)
            dB = planet.squareDist(sunB_current)
            dC = planet.squareDist(sunC_current)
            illumination = (lA/dA) + (lB/dB) + (lC/dC)

            #check for scorching or freezing
            if illumination > scoeff:
                return steps * dt
            if illumination < fcoeff:
                return steps * dt
            steps += 1
        return 10.01

        
    
class Simulation:
    # Default simulation time 10yr, step time 0.01yr
    def __init__(self, Bodies, total_time = 10, dt=0.01):
        self.bodies = Bodies
        self.total_time = total_time
        self.dt = dt
        self.timesteps = int(total_time/dt)
        
    # Runs the simulation and produces an array of arrays of Bodies.
    # The t-th entry in the list is the position of Bodies after the t-th timestep
    # has been simulated    
    def run(self):
        pss = [None]*(self.timesteps+1)
        pss[0] = self.bodies
        for t in range(self.timesteps):
            # for every Body in the current timestep add its next position in next timestep
            pss[t+1] = [pss[t][i].next(pss[t],self.dt) for i in range(len(self.bodies))]
        return pss
    
    def closestDistance(self): # 15%
        x = self.run()
        # if there are less than 2 bodies, return None
        count = 0
        for _ in x[0]:
            count += 1

        if count < 2:
            return None
        
        
        #to calculate closest distance
        min_distance = float('inf')
        for t in range(self.timesteps+1):
            count2 = 0
            for _ in x[t]:
                count2 += 1
            for i in range(count2):
                for j in range(i+1, count2):
                    dist = math.sqrt(x[t][i].squareDist(x[t][j]))
                    if dist < min_distance:
                        min_distance = dist
        return min_distance

        
    
    # Shows an animation of the simulation using matplotlib
    def show(self,x0,y0,x1,y1):
        pss = self.run()
        # get figure and axes objects
        fig, ax = plt.subplots()
        # set some reasonable zoom on axes
        ax.set_xlim(x0,x1)
        ax.set_ylim(y0,y1)
        # put labels on the window for Bodies with different colours
        scatter = []
        for i in range(len(self.bodies)):
            scatter.append(ax.scatter([], [], marker='o', label=f'Body {i}'))
         # add timestep text to the legend
        time_text = ax.text(0.02, 0.98, '', transform=ax.transAxes, 
                       verticalalignment='top', bbox=dict(boxstyle='round', 
                       facecolor='wheat', alpha=0.5))
        # update function from one position to the next
        def update(frame):
            for i in range(len(self.bodies)):
                scatter[i].set_offsets([pss[frame][i].x, pss[frame][i].y])
            time_text.set_text(f'Timestep: {frame}')                
            return scatter + [time_text]
        # generate and show the animation
        a = FuncAnimation(fig, update, frames=len(pss), interval=1, blit=True, repeat=False)
        plt.xlabel("X coordinate (AU)")
        plt.ylabel("Y coordinate (AU)")
        plt.title("Celestial body trajectories")
        plt.legend()
        plt.show()    

In [60]:
####################################################################
# Question 2: The tree Gadget
# To solve: add and remove elements on a Gadget
####################################################################

#GNode can have 3 states, an empty leaf, a leaf containing exactly 1 body, or an internal node with 4 children
class Gadget:
    def __init__(self,box):
        self.root = GNode(box)
        self.size = 0
        
    # Add Body p to this Gadget [30% TODO]        
    def add(self,p):

        #Helper function to determine which quadrant of a node a body p belongs to
        #(NE-0,NW-1,SW-2,SE-3)
        def getQuadrant(node, p):
            if p.x >= node.box.mx and p.y >= node.box.my:
                return 0
            elif p.x < node.box.mx and p.y >= node.box.my:
                return 1
            elif p.x < node.box.mx and p.y < node.box.my:
                return 2
            elif p.x >= node.box.mx and p.y < node.box.my:
                return 3

        currentNode = self.root
        stack1 = Stack()

        while currentNode.children is not None:
            stack1.push(currentNode)
            quadrant = getQuadrant(currentNode, p)
            currentNode = currentNode.children[quadrant]

        #Case 1: Empty leaf node
        if currentNode.p is None and currentNode.nbodies == 0: #Empty leaf node
            currentNode.p = p
            currentNode.nbodies = 1
            currentNode.updateCOM()

            #Bubble up through the stack to update rest of n bodies 
            while not stack1.isEmpty():
                nodeToUpdate = stack1.pop()
                nodeToUpdate.nbodies += 1
                nodeToUpdate.updateCOM()

            self.size +=1
            return
        
        #Case 2: Leaf node with 1 body
        elif currentNode.p is not None and currentNode.nbodies == 1: #Leaf node with 1 body
            insertedFlag = False
            
            while insertedFlag == False:

                #Save the current Body stored in Node as we will need to reinsert it
                oldbody = currentNode.p
                currentNode.p = None

                #Split the node into 4 children
                bNE, bNW, bSW, bSE = currentNode.box.split4()
                currentNode.children = [GNode(bNE), GNode(bNW), GNode(bSW), GNode(bSE)]

                #Insert the old body into the correct child/quadrant
                oldbodyQuadrant = getQuadrant(currentNode, oldbody)
                oldbodyNode = currentNode.children[oldbodyQuadrant]
                oldbodyNode.p = oldbody
                oldbodyNode.nbodies = 1
                oldbodyNode.updateCOM()

                stack1.push(currentNode)
                #Now we move on to inserting body p into the right position
                currentNode = currentNode.children[getQuadrant(currentNode, p)]

                #If the child is empty, insert p and stop
                if currentNode.p is None and currentNode.nbodies == 0:
                    currentNode.p = p
                    currentNode.nbodies = 1
                    currentNode.updateCOM()
                    insertedFlag = True
                    break
                    
                #Continue the loop until we have found an empty node and can insert p without splitting further
                #This is to combat the case that the old body and new body fall into the same quadrant
                if currentNode.p is not None and currentNode.nbodies == 1:
                    continue

            #Bubble through the stack to update rest of n bodies
            #This bubble up is more complicated than case A due to the redistribution of bodies
            #We need to recalculate the number of bodies in each node
            while stack1.isEmpty() == False:
                nodeToUpdate = stack1.pop()
                total = 0
                for child in nodeToUpdate.children:
                    total += child.nbodies

                nodeToUpdate.nbodies = total
                nodeToUpdate.updateCOM()
                
            self.size +=1
            return
        
    # Remove Body p from this Gadget [20% TODO]
    def remove(self,p): 
        #Helper function to determine which quadrant of a node a body p belongs to
        #(NE-0,NW-1,SW-2,SE-3)
        def getQuadrant(node, p):
            if p.x >= node.box.mx and p.y >= node.box.my:
                return 0
            elif p.x < node.box.mx and p.y >= node.box.my:
                return 1
            elif p.x < node.box.mx and p.y < node.box.my:
                return 2
            elif p.x >= node.box.mx and p.y < node.box.my:
                return 3
            
        currentNode = self.root
        stack1 = Stack()

        #Traverse the tree to the leaf node
        while currentNode.children is not None:
            stack1.push(currentNode)
            quadrant = getQuadrant(currentNode, p)
            currentNode = currentNode.children[quadrant]

        #If the body is found in the leaf node then remove it
        if currentNode.p == p:
            currentNode.p = None
            currentNode.nbodies = 0
            currentNode.updateCOM()
            self.size -= 1

        else: #Body not found, exit the method
            return
        
        #Bubble up through the stack to update rest of nbodies
        while stack1.isEmpty() == False:
            nodeToUpdate = stack1.pop()
            singleBody = None #Used to store the single body found if we need to collapse an internal node into a leaf node with a single body
            total = 0

            for child in nodeToUpdate.children:
                total += child.nbodies
                #Check if there is a single body in any of the children in case we need to collapse later
                if child.nbodies == 1 and child.p is not None:
                    singleBody = child.p

            #Collapse to an empty leaf node
            if total == 0:
                nodeToUpdate.children = None
                nodeToUpdate.p = None
                nodeToUpdate.nbodies = 0
                nodeToUpdate.updateCOM()
            
            #Collapse to a leaf node with 1 body
            elif total == 1:
                nodeToUpdate.children = None
                nodeToUpdate.p = singleBody
                nodeToUpdate.nbodies = 1
                nodeToUpdate.updateCOM()
            
            #Still an internal node
            else:
                nodeToUpdate.nbodies = total
                nodeToUpdate.updateCOM()


    def __str__(self):
        return self.root.niceStr()

    # Collect all Bodies from this Gadget and return them in an array
    def getBodies(self):
        A = [None]*self.size
        i = 0; stack = Stack(); stack.push(self.root)
        while not stack.isEmpty():
            n = stack.pop()
            if n.isLeaf():
                if n.p is not None:
                    A[i] = n.p
                    i += 1
            else:
                for c in n.children: stack.push(c)
        return A   

    # Build a new Gadget and add in it all Bodies from array ps
    @staticmethod
    def fromBodies(ps):
        # calculate bottom-left and top-right positions
        if ps == []: return None
        x0 = x1 = ps[0].x
        y0 = y1 = ps[0].y
        for p in ps:
            if p.x < x0: x0 = p.x
            elif p.x > x1: x1 = p.x
            if p.y < y0: y0 = p.y
            elif p.y > y1: y1 = p.y
        # build gadget and add bodies 
        g = Gadget(Box(x0,y0,x1,y1))
        for p in ps: g.add(p)
        return g    
    
    @staticmethod
    def _drawNode(ax, node, showCom):
        x0, y0, x1, y1, mx, my = node.box.asTuple()
        w = x1-x0; h = y1-y0
        # draw the node rectangle
        rect = Rectangle((x0, y0), w, h, fill=False, linewidth=0.8, alpha=0.5)
        ax.add_patch(rect)
        # draw COM for internal nodes
        if showCom and not node.isLeaf():
            ax.plot([node.COM.x], [node.COM.y], marker='+', markersize=6)
            ax.plot([mx, node.COM.x], [my, node.COM.y], linewidth=0.3)            
        # draw leaf contents
        if node.isLeaf(): 
            if node.p is not None:
                ax.plot([node.p.x], [node.p.y], marker='o', markersize=3)
        else:
            # recurse into children [NE, NW, SW, SE]
            for c in node.children:
                if c is not None:
                    Gadget._drawNode(ax, c, showCom)
                
    def plot(self, figsize=(6,6)): # figsize : (w,h) in inches
        showCom=True         # Draw line with '+' from center to COM of internal nodes
        margin_ratio=0.05    # Extra margin around the root box
        fig, ax = plt.subplots(figsize=figsize)
        # Plot node bounds
        Gadget._drawNode(ax, self.root, showCom)
        # collect Bodies from leaves
        A = self.getBodies()
        if A != []:
            xs, ys = zip(*[(A[i].x,A[i].y) for i in range(len(A))])
            ax.plot(xs, ys, linestyle='none', marker='o', markersize=3)
        # Set view limits from root box (with margin)
        x0, y0, x1, y1, mx, my = self.root.box.asTuple()
        dx, dy = x1-x0, y1-y0
        pad_x = margin_ratio * max(dx, 1e-12)
        pad_y = margin_ratio * max(dy, 1e-12)
        ax.set_xlim(x0-pad_x, x1+pad_x)
        ax.set_ylim(y0-pad_y, y1+pad_y)
        ax.set_aspect('equal', adjustable='box')
        plt.tight_layout()
        plt.show()

In [66]:
####################################################################
# Question 3: Using the Gadget to speed up simulations
# To solve: get bodies from a Gadget wrt reference body 
####################################################################

class FastSimulation(Simulation):
    # Runs the simulation and produces an array of arrays of Bodies.
    # The t-th entry in the list is the position of Bodies after the t-th timestep
    # has been simulated.
    # Simulation uses a Gadget in order to approximate for each body the list 
    # of other bodies gravitationally affecting its trajectory.
    def run(self,test=None): 
        pss = [None]*(self.timesteps+1)
        pss[0] = self.bodies
        for t in range(self.timesteps):
            # calculate the current Gadget
            g = Gadget.fromBodies(pss[t])
            # for every Body in the current timestep add its next position in next timestep
            # but using the gadget g
            A = pss[t][:]
            for i in range(len(A)):
                new_ps = FastSimulation.getBodies(g,A[i])
                A[i] = A[i].next(new_ps,self.dt)
                if test is not None: test[i] = new_ps
            pss[t+1] = A
        return pss

    # Barnes-Hut criterion deciding whether node n should be opened when calculating
    # the gravitational forces exerted on body p.
    # By defualt, if p is in n then we open.
    @staticmethod
    def BarnesHut(n,p):
        theta = 0.7
        if n.box.isIn(p): return True
        sqDist = (n.COM.x-p.x)**2+(n.COM.y-p.y)**2
        return n.box.maxSide**2/sqDist >= theta**2
        
    # Get bodies from gadget g in an optimal way, using criterion shouldOpen and body p to 
    # decide whether nodes in the gadget should be "opened" (i.e. their subnodes examined)
    # or not (i.e. the whole subtree approximated by its COM).
    
    @staticmethod
    def getBodies(g, p, shouldOpen=BarnesHut):  # 20%
        #Depth first traversal is done using a stack, pushing traversable nodes onto the stack
        #This functions basically returns an array of all the bodies in G except where an internal node of 4 bodies is far
        #enough where it can be treated as 1 body, in which case all of it's children are ignored as well
        #Have to change the default argument from shouldOpen=BarnesHut to shouldOpen=None to avoid 'object is not callable errors' in Python 3.9.7 in VSCode
        #This function stores the bodies in a stack to maintain an effienecy better than python lists (O(n)^2)
        
        #if shouldOpen is None:
        #   shouldOpen = FastSimulation.BarnesHut

        new_ps = Stack()
        stack1 = Stack()
        
        #In the case that gadget is empty, return empty list
        if g is None or g.root is None:
            return new_ps.toArray()

        stack1.push(g.root)

        #While we have nodes to traverse/process
        while stack1.isEmpty() == False:
            currentNode = stack1.pop()

            #This line is technically not needed for the code to function but is good for efficiency
            if currentNode.nbodies == 0:
                continue

            #Check if it is a leaf node
            if currentNode.isLeaf():
                if currentNode.p is not None:
                    new_ps.push(currentNode.p)
            
            #If it is an internal Node, check if we should open it
            elif shouldOpen(currentNode, p) == True:
                for child in currentNode.children:
                    if child is not None and child.nbodies > 0:
                        stack1.push(child)
            
            #Internal node doesn't need to be opened therefore just use COM
            else:
                new_ps.push(currentNode.COM)

        return new_ps.toArray()
